In [4]:
from pyspark import SparkConf, SparkContext

In [5]:
from operator import add

In [6]:
conf = (SparkConf()
        .setMaster("local")
        .setAppName("Desafio")
        .set("spark.executor.memory","4g"))


In [ ]:
sc = SparkContext(conf = conf)


In [13]:
julho = sc.textFile('NASA_access_log_Jul95')
julho = julho.cache()

In [14]:
agosto = sc.textFile('NASA_access_log_Aug95')
agosto = agosto.cache()

In [ ]:
agosto

## Contagem de valores unicos


In [ ]:
count_agosto_host_distinc = agosto.flatMap(lambda linha: linha.split(' ')[0]).distinct().count()
print('Valores unicos contado em agosto %s' %count_agosto_host_distinc)

count_julho_host_distinc = julho.flatMap(lambda linha: linha.split(' ')[0]).distinct().count()
print('Valores unicos contado em julho %s' %count_julho_host_distinc)

## Contagem de erros 404


In [ ]:
def total_404(linha):
    try:
        code = linha.split(' ')[-2]
        if code == '404':
            return True
    except:
        pass
    
    return False

total_agosto_404 = agosto.filter(total_404).cache()
total_julho_404 = julho.flatMap(lambda linha: linha.split(' ')[-2] == '404').cache()

print('Total de Erros 404 contados em agosto %s' %total_agosto_404.count())
print('Total de Erros 404 contados em julho %s' %total_julho_404.count())

## Top 5 Urls

In [ ]:
def top5erros(rdd):
    erro = rdd.map(lambda linha: linha.splt('"')[1].split(' ')[1])
    contador = erro.map(lambda erro: (erro,1)).reduceByKey (add)
    top = contador.sortBy(lambda pair: -pair[1].take(5))
    
    print('\n Top 5 urls com mais frequencia sao: ')
    for erro, count in top:
        print(erro, count)
    
    return top

top5erros(total_agosto_404)
top5erros(total_julho_404)

## Quantidade por dia

In [ ]:

def dia_count(rdd):
    dias = rdd.map(lambda linha: linha.split('[')[1].split(':')[0])
    contador = dias.map(lambda dia: (dia, 1)).reduceByKey(add).collect()
    
    print('\nQuantidade por dia')
    for dia, count in contador:
        print(dia, count)
        
    return contador

contador_dia(julho_404)
contador_dia(agosto_404)

## Total de bytes Retornados

In [ ]:
def bytes_retornados(rdd):
    def byte_count(line):
        try:
            count = int(line.split(" ")[-1])
            if count < 0:
                raise ValueError()
            return count
        except:
            return 0
        
    count = rdd.map(byte_count).reduce(add)
    return count

print('O total de bytes retornados em Julho: %s' % count_dados_acumulados(julho))
print('O total de bytes retornados em Agosto: %s' % count_dados_acumulados(agosto))